# Debugging scikit-learn text classification pipeline

scikit-learn docs provide a nice text classification [tutorial](http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html). Make sure to read it first. We'll be doing something similar to it, while taking more detailed look at classifier weights and predictions.

## 1. Baseline model

First, we need some data. Let's load 20 Newsgroups data, keeping only 4 categories:

In [1]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'soc.religion.christian', 
              'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(
    subset='train',
    categories=categories,
    shuffle=True,
    random_state=42
)
twenty_test = fetch_20newsgroups(
    subset='test',
    categories=categories,
    shuffle=True,
    random_state=42
)

A basic text processing pipeline - bag of words features and Logistic Regression as a classifier:

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.pipeline import make_pipeline

vec = CountVectorizer()
clf = LogisticRegressionCV()
pipe = make_pipeline(vec, clf)
pipe.fit(twenty_train.data, twenty_train.target);

We're using LogisticRegressionCV here to adjust regularization
parameter C automatically. It allows to compare different
vectorizers - optimal C value could be different for different input
features (e.g. for bigrams or for character-level input). An alternative
would be to use GridSearchCV or RandomizedSearchCV.


Let's check quality of this pipeline:

In [3]:
from sklearn import metrics

def print_report(pipe):
    y_test = twenty_test.target
    y_pred = pipe.predict(twenty_test.data)
    report = metrics.classification_report(y_test, y_pred, 
        target_names=twenty_test.target_names)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))
    
print_report(pipe)

                        precision    recall  f1-score   support

           alt.atheism       0.93      0.80      0.86       319
         comp.graphics       0.87      0.96      0.91       389
               sci.med       0.94      0.81      0.87       396
soc.religion.christian       0.85      0.98      0.91       398

           avg / total       0.90      0.89      0.89      1502

accuracy: 0.891


Not bad. We can try other classifiers and preprocessing methods, but let's check first what the model learned using eli5.explain_weights function:

In [4]:
import eli5
eli5.explain_weights(clf, top=10)

Explanation(estimator="LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,\n           fit_intercept=True, intercept_scaling=1.0, max_iter=100,\n           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,\n           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)", description="\nFeatures with largest coefficients per class.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target=0, feature_weights=FeatureWeights(pos=[FeatureWeight(feature='x21167', weight=1.9913874089700465, std=None), FeatureWeight(feature='x19218', weight=1.9246200045966999, std=None), FeatureWeight(feature='x5714', weight=1.8342044762229246, std=None), FeatureWeight(feature='x23677', weight=1.8132954104985863, std=None), FeatureWeight(feature='x15511', weight=1.6971355589877903, std=None), FeatureWeight(feature='x26415', weight=1.6959151064421698, std=None), FeatureWeight(feature='x6440', weight=1.617482489999843, std=None), FeatureWeight(feature='x26412', weight=1.5935095786431324, std=None)], neg=[FeatureWeight(feature='<BIAS>', weight=-10.452679661739383, std=None), FeatureWeight(feature='x28473', weight=-1.68605055687886, std=None)], pos_remaining=10174, neg_remaining=25605), proba=None, score=None, weighted_spans=None), TargetExplanation(target=1, feature_weights=FeatureWeights(pos=[FeatureWeight(feature='x15699', weight=1.7015909636743556, std=None), FeatureWeight(feature='x17366', weight=0.82509541849799761, std=None), FeatureWeight(feature='x14281', weight=0.79767325900407582, std=None), FeatureWeight(feature='x30117', weight=0.78626612548967689, std=None), FeatureWeight(feature='x14277', weight=0.77870707477276635, std=None), FeatureWeight(feature='x17356', weight=0.77291881847879196, std=None), FeatureWeight(feature='x24267', weight=0.72909661816870341, std=None), FeatureWeight(feature='x7874', weight=0.72424030940699147, std=None), FeatureWeight(feature='x2148', weight=0.7024458565736913, std=None)], neg=[FeatureWeight(feature='<BIAS>', weight=-1.3787488993070751, std=None)], pos_remaining=11710, neg_remaining=24069), proba=None, score=None, weighted_spans=None), TargetExplanation(target=2, feature_weights=FeatureWeights(pos=[FeatureWeight(feature='x25234', weight=2.0156519386230851, std=None), FeatureWeight(feature='x12026', weight=1.9510879311806237, std=None), FeatureWeight(feature='x17854', weight=1.7582913242082159, std=None), FeatureWeight(feature='x11729', weight=1.6970297014890874, std=None), FeatureWeight(feature='x32847', weight=1.6548284065436876, std=None), FeatureWeight(feature='x22379', weight=1.5223415043834094, std=None), FeatureWeight(feature='x16328', weight=1.5182910331759258, std=None)], neg=[FeatureWeight(feature='<BIAS>', weight=-5.012815833941537, std=None), FeatureWeight(feature='x15699', weight=-2.1705331812754349, std=None), FeatureWeight(feature='x15521', weight=-1.7637394419424461, std=None)], pos_remaining=15007, neg_remaining=20772), proba=None, score=None, weighted_spans=None), TargetExplanation(target=3, feature_weights=FeatureWeights(pos=[FeatureWeight(feature='x28473', weight=1.1933581281233412, std=None), FeatureWeight(feature='x8609', weight=1.0299533608714988, std=None), FeatureWeight(feature='x8559', weight=1.0214428133992981, std=None), FeatureWeight(feature='x8798', weight=0.94568841384121582, std=None), FeatureWeight(feature='x8544', weight=0.89888735105870832, std=None), FeatureWeight(feature='x8553', weight=0.79678024883367982, std=None)], neg=[FeatureWeight(feature='x16881'

The table above doesn't make any sense; the problem is that eli5 was not able to get feature and class names from the classifier object alone. We can provide feature and target names explicitly:

In [5]:
# eli5.explain_weights(clf, 
#                      feature_names=vec.get_feature_names(), 
#                      target_names=twenty_test.target_names)

The code above works, but a better way is to provide vectorizer instead and let eli5 figure out the details automatically:

In [6]:
eli5.explain_weights(clf, vec=vec, top=10, 
                     target_names=twenty_test.target_names)

Explanation(estimator="LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,\n           fit_intercept=True, intercept_scaling=1.0, max_iter=100,\n           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,\n           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)", description="\nFeatures with largest coefficients per class.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='alt.atheism', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='mathew', weight=1.9913874089700465, std=None), FeatureWeight(feature='keith', weight=1.9246200045966999, std=None), FeatureWeight(feature='atheism', weight=1.8342044762229246, std=None), FeatureWeight(feature='okcforum', weight=1.8132954104985863, std=None), FeatureWeight(feature='go', weight=1.6971355589877903, std=None), FeatureWeight(feature='psuvm', weight=1.6959151064421698, std=None), FeatureWeight(feature='believing', weight=1.617482489999843, std=None), FeatureWeight(feature='psu', weight=1.5935095786431324, std=None)], neg=[FeatureWeight(feature='<BIAS>', weight=-10.452679661739383, std=None), FeatureWeight(feature='rutgers', weight=-1.68605055687886, std=None)], pos_remaining=10174, neg_remaining=25605), proba=None, score=None, weighted_spans=None), TargetExplanation(target='comp.graphics', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='graphics', weight=1.7015909636743556, std=None), FeatureWeight(feature='images', weight=0.82509541849799761, std=None), FeatureWeight(feature='files', weight=0.79767325900407582, std=None), FeatureWeight(feature='software', weight=0.78626612548967689, std=None), FeatureWeight(feature='file', weight=0.77870707477276635, std=None), FeatureWeight(feature='image', weight=0.77291881847879196, std=None), FeatureWeight(feature='package', weight=0.72909661816870341, std=None), FeatureWeight(feature='card', weight=0.72424030940699147, std=None), FeatureWeight(feature='3d', weight=0.7024458565736913, std=None)], neg=[FeatureWeight(feature='<BIAS>', weight=-1.3787488993070751, std=None)], pos_remaining=11710, neg_remaining=24069), proba=None, score=None, weighted_spans=None), TargetExplanation(target='sci.med', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='pitt', weight=2.0156519386230851, std=None), FeatureWeight(feature='doctor', weight=1.9510879311806237, std=None), FeatureWeight(feature='information', weight=1.7582913242082159, std=None), FeatureWeight(feature='disease', weight=1.6970297014890874, std=None), FeatureWeight(feature='treatment', weight=1.6548284065436876, std=None), FeatureWeight(feature='msg', weight=1.5223415043834094, std=None), FeatureWeight(feature='health', weight=1.5182910331759258, std=None)], neg=[FeatureWeight(feature='<BIAS>', weight=-5.012815833941537, std=None), FeatureWeight(feature='graphics', weight=-2.1705331812754349, std=None), FeatureWeight(feature='god', weight=-1.7637394419424461, std=None)], pos_remaining=15007, neg_remaining=20772), proba=None, score=None, weighted_spans=None), TargetExplanation(target='soc.religion.christian', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='rutgers', weight=1.1933581281233412, std=None), FeatureWeight(feature='church', weight=1.0299533608714988, std=None), FeatureWeight(feature='christians', weight=1.0214428133992981, std=None), FeatureWeight(feature='clh', weight=0.94568841384121582, std=None), FeatureWeight(feature='christ', weight=0.89888735105870832, std=None), FeatureWeight(feature='christian', weight=0

This starts to make more sense. Columns are target classes. In each column there are features and their weights. Intercept (bias) feature is shown as ``<BIAS>`` in the same table. We can inspect features and weights because we're using a bag-of-words vectorizer and a linear classifier (so there is a direct mapping between individual words and classifier coefficients). For other classifiers features can be harder to inspect.

Some features look good, but some don't. It seems model learned some names specific to a dataset (email parts, etc.) though, instead of learning topic-specific words. Let's check prediction results on an example:

In [7]:
eli5.explain_prediction(clf, twenty_test.data[0], vec=vec, 
                        target_names=twenty_test.target_names)

Weight,Feature
+1.743,Highlighted in text (sum)
-10.453,<BIAS>
Weight,Feature
-1.379,<BIAS>
-3.213,Highlighted in text (sum)
Weight,Feature
+8.958,Highlighted in text (sum)
-5.013,<BIAS>
Weight,Feature
-0.258,<BIAS>


What can be highlighted in text is highlighted in text. There is also a separate table for features which can't be highlighted in text - ``<BIAS>`` in this case. If you hover mouse on a highlighted word it shows you a weight of this word in a title. Words are colored according to their weights.

## 2. Baseline model, improved data

Aha, from the highlighting above it can be seen that a classifier learned some non-interesting stuff indeed, e.g. it remembered parts of email addresses. We should probably clean the data first to make it more interesting; improving model (trying different classifiers, etc.) doesn't make sense at this point - it may just learn to leverage these email addresses better. 

In practice we'd have to do cleaning yourselves; in this example 20 newsgroups dataset provides an option to remove footers and headers from the messages. Nice. Let's clean up the data and re-train a classifier.

In [8]:
twenty_train = fetch_20newsgroups(
    subset='train',
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=['headers', 'footers'],
)
twenty_test = fetch_20newsgroups(
    subset='test',
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=['headers', 'footers'],
)

vec = CountVectorizer()
clf = LogisticRegressionCV()
pipe = make_pipeline(vec, clf)
pipe.fit(twenty_train.data, twenty_train.target);

We just made the task harder and more realistic for a classifier.

In [9]:
print_report(pipe)

                        precision    recall  f1-score   support

           alt.atheism       0.83      0.78      0.80       319
         comp.graphics       0.82      0.96      0.88       389
               sci.med       0.89      0.80      0.84       396
soc.religion.christian       0.88      0.86      0.87       398

           avg / total       0.85      0.85      0.85      1502

accuracy: 0.852


A great result - we just made quality worse! Does it mean pipeline is worse now? No, likely it has a better quality on unseen messages. It is evaluation which is more fair now. Inspecting features used by classifier allowed us to notice a problem with the data and made a good change, despite of numbers which told us not to do that.

Instead of removing headers and footers we could have improved evaluation setup directly, using e.g. GroupKFold from scikit-learn. Then quality of old model would have dropped, we could have removed headers/footers and see increased accuracy, so the numbers would have told us to remove headers and footers. It is not obvious how to split data though, what groups to use with GroupKFold.

So, what have the updated classifier learned? (output is less verbose because only a subset of classes is shown - see "targets" argument):

In [10]:
eli5.explain_prediction(clf, twenty_test.data[0], vec=vec, 
                        target_names=twenty_test.target_names,
                        targets=['sci.med'])

Weight,Feature
+1.747,Highlighted in text (sum)
-1.716,<BIAS>


Hm, it no longer uses email addresses, but it still doesn't look good: classifier assigns high weights to seemingly unrelated words like 'do' or 'my'. These words appear in many texts, so maybe classifier uses them as a proxy for bias. Or maybe some of them are more common in some of classes. 

## 3. Pipeline improvements

To help classifier we may filter out stop words:

In [11]:
vec = CountVectorizer(stop_words='english')
clf = LogisticRegressionCV()
pipe = make_pipeline(vec, clf)
pipe.fit(twenty_train.data, twenty_train.target)

print_report(pipe)

                        precision    recall  f1-score   support

           alt.atheism       0.87      0.76      0.81       319
         comp.graphics       0.85      0.95      0.90       389
               sci.med       0.93      0.85      0.89       396
soc.religion.christian       0.85      0.89      0.87       398

           avg / total       0.87      0.87      0.87      1502

accuracy: 0.871


In [12]:
eli5.explain_prediction(clf, twenty_test.data[0], vec=vec, 
                        target_names=twenty_test.target_names,
                        targets=['sci.med'])

Weight,Feature
+2.184,Highlighted in text (sum)
-1.674,<BIAS>


Looks better, isn't it? 

Alternatively, we can use TF\*IDF scheme; it should give a somewhat similar effect. 

Note that we're cross-validating LogisticRegression regularisation parameter here, like in other examples (LogisticRegressionCV, not LogisticRegression). TF\*IDF values are different from word count values, so optimal C value can be different. We could draw a wrong conclusion if a classifier with fixed regularization strength is used - the chosen C value could have worked better for one kind of data.

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer()
clf = LogisticRegressionCV()
pipe = make_pipeline(vec, clf)
pipe.fit(twenty_train.data, twenty_train.target)

print_report(pipe)

                        precision    recall  f1-score   support

           alt.atheism       0.91      0.79      0.85       319
         comp.graphics       0.83      0.97      0.90       389
               sci.med       0.95      0.87      0.91       396
soc.religion.christian       0.90      0.91      0.91       398

           avg / total       0.90      0.89      0.89      1502

accuracy: 0.892


In [14]:
eli5.explain_prediction(clf, twenty_test.data[0], vec=vec, 
                        target_names=twenty_test.target_names,
                        targets=['sci.med'])

Weight,Feature
+6.788,Highlighted in text (sum)
-5.203,<BIAS>


It helped, but didn't have quite the same effect. Why not do both?

In [15]:
vec = TfidfVectorizer(stop_words='english')
clf = LogisticRegressionCV()
pipe = make_pipeline(vec, clf)
pipe.fit(twenty_train.data, twenty_train.target)

print_report(pipe)

                        precision    recall  f1-score   support

           alt.atheism       0.93      0.77      0.84       319
         comp.graphics       0.84      0.97      0.90       389
               sci.med       0.95      0.89      0.92       396
soc.religion.christian       0.88      0.92      0.90       398

           avg / total       0.90      0.89      0.89      1502

accuracy: 0.893


In [16]:
eli5.explain_prediction(clf, twenty_test.data[0], vec=vec, 
                        target_names=twenty_test.target_names,
                        targets=['sci.med'])

Weight,Feature
+5.488,Highlighted in text (sum)
-3.578,<BIAS>


This starts to look good! 


## 4. Char-based pipeline

Maybe we can get somewhat better quality by choosing a different classifier, but let's skip it for now. Let's try other analysers instead - use char n-grams instead of words:

In [17]:
vec = TfidfVectorizer(stop_words='english', analyzer='char', 
                      ngram_range=(3,5))
clf = LogisticRegressionCV()
pipe = make_pipeline(vec, clf)
pipe.fit(twenty_train.data, twenty_train.target)

print_report(pipe)

                        precision    recall  f1-score   support

           alt.atheism       0.93      0.79      0.85       319
         comp.graphics       0.81      0.97      0.89       389
               sci.med       0.95      0.86      0.90       396
soc.religion.christian       0.89      0.91      0.90       398

           avg / total       0.89      0.89      0.89      1502

accuracy: 0.888


In [18]:
eli5.explain_prediction(clf, twenty_test.data[0], vec=vec, 
                        target_names=twenty_test.target_names)

Weight,Feature
-0.838,Highlighted in text (sum)
-6.480,<BIAS>
Weight,Feature
+0.934,<BIAS>
-6.052,Highlighted in text (sum)
Weight,Feature
+4.493,Highlighted in text (sum)
-5.149,<BIAS>
Weight,Feature
+0.600,Highlighted in text (sum)


It works, but quality is a bit worse. Also, it takes ages to train. 

It looks like stop_words have no effect now - in fact, this is documented in scikit-learn docs, so our stop_words='english' was useless. But at least it is now more obvious how the text looks like for a char ngram-based classifier. Grab a cup of tea and see how char_wb looks like:

In [19]:
vec = TfidfVectorizer(analyzer='char_wb', ngram_range=(3,5))
clf = LogisticRegressionCV()
pipe = make_pipeline(vec, clf)
pipe.fit(twenty_train.data, twenty_train.target)

print_report(pipe)

                        precision    recall  f1-score   support

           alt.atheism       0.93      0.79      0.85       319
         comp.graphics       0.87      0.96      0.91       389
               sci.med       0.91      0.90      0.90       396
soc.religion.christian       0.89      0.91      0.90       398

           avg / total       0.90      0.89      0.89      1502

accuracy: 0.894


In [20]:
eli5.explain_prediction(clf, twenty_test.data[0], vec=vec, 
                        target_names=twenty_test.target_names)

Weight,Feature
-2.560,Highlighted in text (sum)
-6.318,<BIAS>
Weight,Feature
+0.974,<BIAS>
-6.981,Highlighted in text (sum)
Weight,Feature
+2.134,Highlighted in text (sum)
-2.573,<BIAS>
Weight,Feature
+3.263,Highlighted in text (sum)


The result is similar, with some minor changes. Quality is better for unknown reason; maybe cross-word dependencies are not that important. 

## 5. Debugging HashingVectorizer

To check that we can try fitting word n-grams instead of char n-grams. But let's deal with efficiency first.
To handle large vocabularies we can use HashingVectorizer from scikit-learn; to make training faster we can employ SGDCLassifier:

In [21]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vec = HashingVectorizer(stop_words='english', ngram_range=(1,2))
clf = SGDClassifier(n_iter=10, random_state=42)
pipe = make_pipeline(vec, clf)
pipe.fit(twenty_train.data, twenty_train.target)

print_report(pipe)

                        precision    recall  f1-score   support

           alt.atheism       0.90      0.80      0.85       319
         comp.graphics       0.88      0.96      0.92       389
               sci.med       0.93      0.90      0.92       396
soc.religion.christian       0.89      0.91      0.90       398

           avg / total       0.90      0.90      0.90      1502

accuracy: 0.899


It was super-fast! We're not choosing regularization parameter using cross-validation though. 
Let's check what model learned:

In [22]:
eli5.explain_prediction(clf, twenty_test.data[0], vec=vec, 
                        target_names=twenty_test.target_names,
                        targets=['sci.med'])

Weight,Feature
+0.678,Highlighted in text (sum)
-0.581,<BIAS>


Result looks similar to CountVectorizer. But with HashingVectorizer we don't even have a vocabulary! Why does it work?

In [23]:
eli5.explain_weights(clf, vec=vec, top=10,
                     target_names=twenty_test.target_names)

Explanation(estimator="SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,\n       eta0=0.0, fit_intercept=True, l1_ratio=0.15,\n       learning_rate='optimal', loss='hinge', n_iter=10, n_jobs=1,\n       penalty='l2', power_t=0.5, random_state=42, shuffle=True, verbose=0,\n       warm_start=False)", description="\nFeatures with largest coefficients per class.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='alt.atheism', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='x199378', weight=2.8355778388548112, std=None), FeatureWeight(feature='x938889', weight=2.3782441808553818, std=None), FeatureWeight(feature='x718537', weight=1.7758722596103569, std=None), FeatureWeight(feature='x349126', weight=1.6253813228096212, std=None), FeatureWeight(feature='x242643', weight=1.5535870143610482, std=None), FeatureWeight(feature='x71928', weight=1.5087186198297369, std=None)], neg=[FeatureWeight(feature='x641063', weight=-2.1316758445142141, std=None), FeatureWeight(feature='x199709', weight=-1.8722520882696891, std=None), FeatureWeight(feature='x741207', weight=-1.79502002499737, std=None), FeatureWeight(feature='x683213', weight=-1.6344376723610337, std=None)], pos_remaining=50341, neg_remaining=50567), proba=None, score=None, weighted_spans=None), TargetExplanation(target='comp.graphics', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='x580586', weight=3.7374454089236568, std=None), FeatureWeight(feature='x342790', weight=2.0556668624828704, std=None), FeatureWeight(feature='x771885', weight=1.9564203049972959, std=None), FeatureWeight(feature='x363686', weight=1.7874391862512466, std=None), FeatureWeight(feature='x111283', weight=1.7172850583980188, std=None)], neg=[FeatureWeight(feature='x814572', weight=-2.4466131778380458, std=None), FeatureWeight(feature='x120354', weight=-2.0208607121102031, std=None), FeatureWeight(feature='x693269', weight=-1.8125946750821811, std=None), FeatureWeight(feature='x85557', weight=-1.7792475552627287, std=None), FeatureWeight(feature='x857427', weight=-1.759542956960227, std=None)], pos_remaining=32081, neg_remaining=31710), proba=None, score=None, weighted_spans=None), TargetExplanation(target='sci.med', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='x988761', weight=2.2092376094416499, std=None), FeatureWeight(feature='x337555', weight=2.1938190227343077, std=None), FeatureWeight(feature='x154565', weight=2.1616903747814122, std=None), FeatureWeight(feature='x806262', weight=1.8177636929969159, std=None)], neg=[FeatureWeight(feature='x494508', weight=-2.1503791390011275, std=None), FeatureWeight(feature='x365313', weight=-2.1466062719974519, std=None), FeatureWeight(feature='x85557', weight=-2.0854002634965094, std=None), FeatureWeight(feature='x34701', weight=-1.8505985280775841, std=None), FeatureWeight(feature='x580586', weight=-1.7501804380992374, std=None), FeatureWeight(feature='x790864', weight=-1.7043511166889813, std=None)], pos_remaining=44124, neg_remaining=43892), proba=None, score=None, weighted_spans=None), TargetExplanation(target='soc.religion.christian', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='x641063', weight=3.0344886348739704, std=None), FeatureWeight(feature='x199709', weight=3.016324431765582, std=None), FeatureWeight(feature='x741207', weight=2.9774676767613868, std=None), FeatureWeight(feature='x396081', weight=2.0921269712655333, std=None), FeatureWeight(feature='x274863', weight=1.9006937867697857, std=None)]

Ok, we don't have a vocabulary, so we don't have feature names. Are we out of luck? Nope, eli5 has an answer for that: InvertableHashingVectorizer. It can be used to get feature names for HahshingVectorizer without fitiing a huge vocabulary. It still needs some data to learn words -> hashes mapping though; we can use a random subset of data to fit it.

In [29]:
from eli5.sklearn import InvertableHashingVectorizer
import numpy as np

In [30]:
ivec = InvertableHashingVectorizer(vec)
sample_size = len(twenty_train.data) // 10
X_sample = np.random.choice(twenty_train.data, size=sample_size)
ivec.fit(X_sample);

In [31]:
eli5.explain_weights(clf, vec=ivec, top=20,
                     target_names=twenty_test.target_names)

Explanation(estimator="SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,\n       eta0=0.0, fit_intercept=True, l1_ratio=0.15,\n       learning_rate='optimal', loss='hinge', n_iter=10, n_jobs=1,\n       penalty='l2', power_t=0.5, random_state=42, shuffle=True, verbose=0,\n       warm_start=False)", description="\nFeatures with largest coefficients per class.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n\nFeature names are restored from their hashes; this is not 100% precise\nbecause collisions are possible. For known collisions possible feature names\nare separated by | sign. Keep in mind the collision list is not exhaustive.\nFeatures marked with (-) should be read as inverted: if they have positive\ncoefficient, the result is negative, if they have negative coefficient,\nthe result is positive.\n", error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='alt.atheism', feature_weights=FeatureWeights(pos=[FeatureWeight(feature=[{'sign': 1.0, 'name': 'atheism'}], weight=2.8355778388548112, std=None), FeatureWeight(feature=[{'sign': 1.0, 'name': 'writes'}], weight=2.3782441808553818, std=None), FeatureWeight(feature=[{'sign': 1.0, 'name': 'morality'}], weight=1.6344376723610337, std=None), FeatureWeight(feature='FEATURE[349126]', weight=1.6253813228096212, std=None), FeatureWeight(feature=[{'sign': 1.0, 'name': 'religion'}], weight=1.5535870143610482, std=None), FeatureWeight(feature=[{'sign': 1.0, 'name': 'islam'}], weight=1.5087186198297369, std=None), FeatureWeight(feature=[{'sign': 1.0, 'name': 'keith'}], weight=1.488661052006395, std=None), FeatureWeight(feature=[{'sign': 1.0, 'name': 'religious'}], weight=1.4760795033187395, std=None), FeatureWeight(feature=[{'sign': 1.0, 'name': 'objective'}], weight=1.4385335668778914, std=None), FeatureWeight(feature=[{'sign': 1.0, 'name': 'wrote'}], weight=1.4140859668917314, std=None), FeatureWeight(feature=[{'sign': 1.0, 'name': 'said'}], weight=1.4046999727036988, std=None), FeatureWeight(feature=[{'sign': 1.0, 'name': 'punishment'}], weight=1.3614734489114138, std=None), FeatureWeight(feature=[{'sign': 1.0, 'name': 'livesey'}], weight=1.3349488885727521, std=None), FeatureWeight(feature=[{'sign': 1.0, 'name': 'heard outcry'}, {'sign': 1.0, 'name': 'mathew'}], weight=1.3316349560231588, std=None), FeatureWeight(feature=[{'sign': 1.0, 'name': 'atheist'}], weight=1.3236649558031317, std=None), FeatureWeight(feature=[{'sign': 1.0, 'name': 'agree'}], weight=1.3201297385790673, std=None)], neg=[FeatureWeight(feature=[{'sign': 1.0, 'name': 'christians'}], weight=-2.1316758445142141, std=None), FeatureWeight(feature=[{'sign': 1.0, 'name': 'christ'}], weight=-1.8722520882696891, std=None), FeatureWeight(feature=[{'sign': 1.0, 'name': 'rutgers'}], weight=-1.79502002499737, std=None), FeatureWeight(feature=[{'sign': 1.0, 'name': 'rutgers edu'}, {'sign': -1.0, 'name': 'previuos'}], weight=-1.7758722596103569, std=None)], pos_remaining=50060, neg_remaining=50838), proba=None, score=None, weighted_spans=None), TargetExplanation(target='comp.graphics', feature_weights=FeatureWeights(pos=[FeatureWeight(feature=[{'sign': 1.0, 'name': 'graphics'}], weight=3.7374454089236568, std=None), FeatureWeight(feature=[{'sign': 1.0, 'name': 'image'}], weight=2.4466131778380458, std=None), FeatureWeight(feature=[{'sign': 1.0, 'name': 'code'}], weight=2.0556668624828704, std=None), FeatureWeight(feature=[{'sign': 1.0, 'name': 'files'}], weight=2.0208607121102031, std=None), FeatureWeight(feature=[{'sign': 1.0, 'name': 'images'}], weight=1.95642030499729

There are collisions (hover mouse over features with "..."), and there are important features which were not seen in the random sample (FEATURE[...]), but overall it looks fine. 

"rutgers edu" bigram feature is suspicious though, it looks like a part of URL.

In [27]:
rutgers_example = [x for x in twenty_train.data if 'rutgers' in x.lower()][0]
print(rutgers_example)

In article <Apr.8.00.57.41.1993.28246@athos.rutgers.edu> REXLEX@fnal.gov writes:
>In article <Apr.7.01.56.56.1993.22824@athos.rutgers.edu> shrum@hpfcso.fc.hp.com
>Matt. 22:9-14 'Go therefore to the main highways, and as many as you find
>there, invite to the wedding feast.'...

>hmmmmmm.  Sounds like your theology and Christ's are at odds. Which one am I 
>to believe?


Yep, it looks like model learned this address instead of learning something useful.

In [28]:
eli5.explain_prediction(clf, rutgers_example, vec=vec, 
                        target_names=twenty_test.target_names, 
                        targets=['soc.religion.christian'])

Weight,Feature
+2.706,Highlighted in text (sum)
-0.662,<BIAS>


Quoted text makes it too easy for model to classify some of the messages; that won't generalize to new messages. So to improve the model next step could be to process the data further, e.g. remove quoted text or replace email addresses with a special token. 

You get the idea: looking at features helps to understand how classifier works. Maybe even more importantly, it helps to notice preprocessing bugs, data leaks, issues with task specification - all these nasty problems you get in a real world.